# Data Mining Homework-3 Task 1

In [38]:
import matplotlib.pyplot as plt
from IPython.display import display, Markdown
import numpy as np
import pandas as pd
from sklearn.metrics.pairwise import pairwise_distances
from sklearn.metrics import accuracy_score

In [39]:
class KMeansClustering:

    def __init__(self, k, stopping_criterion="no_change") -> None:
        self.k = k
        self.stopping_criterion = stopping_criterion
        self.centroids = None
        self._sse_score = None
        self._last_sse_score = float('inf')
        self._iterations = 0

    def euclidean_distance(self, data_point, centroids):
        return np.sqrt(np.sum((centroids - data_point)**2, axis=1))

    def __sum_of_squared_errors_calc(self, centroids, data, y):
        sum_of_errors = 0.0
        for idx, d in enumerate(data):
            sum_of_errors += np.sum((centroids[y[idx]] - d) ** 2)

        return sum_of_errors

    def get_sum_of_squared_error(self):
        return self._sse_score
    def get_iterations_to_converge(self):
        return self._iterations

    def fit(self, X, max_iterations=200):
        self.centroids = np.random.uniform(
            low=np.amin(X, axis=0),
            high=np.amax(X, axis=0),
            size=(self.k, X.shape[1]))

        y = []
        for _ in range(max_iterations):
            y = []
            for data_point in X:

                distances = self.euclidean_distance(
                    data_point=data_point,
                    centroids=self.centroids)
                cluster_num = np.argmin(distances)
                y.append(cluster_num)
            y = np.asarray(y)

            cluster_indices = []

            for idx in range(self.k):
                cluster_indices.append(np.argwhere(y == idx))

            cluster_centers = []

            for i, indices in enumerate(cluster_indices):
                if len(indices) == 0:
                    cluster_centers.append(self.centroids[i])
                else:
                    cluster_centers.append(np.mean(X[indices], axis=0)[0])

            if self.stopping_criterion == "no_change" and np.max(self.centroids - np.array(cluster_centers)) < 1e-3:
                break
            elif self.stopping_criterion == "increase_sse":
                current_sse = self.__sum_of_squared_errors_calc(X, np.array(cluster_centers), y)
                if current_sse > self._last_sse_score:
                    break
                self._last_sse_score = current_sse
            else:
                self.centroids = np.array(cluster_centers)
            self._iterations += 1

        self._sse_score = self.__sum_of_squared_errors_calc(X, self.centroids, y)

        return y

In [40]:
data = np.array(pd.read_csv('data.csv', header=None))
labels = np.ravel(pd.read_csv('label.csv', header=None))
print('Data: ', data.shape)
print('Labels: ', labels.shape)

Data:  (10000, 784)
Labels:  (10000,)


In [41]:
unique_labels = np.unique(labels)
no_of_clusters = unique_labels.size
Max_iterations = 100

In [42]:
euclidean_kmeans = KMeansClustering(k=no_of_clusters)
euclidean_kmeans_labels = euclidean_kmeans.fit(X=data, max_iterations=Max_iterations)

In [43]:
cosine_distances = pairwise_distances(data, metric='cosine')
cosine_kmeans = KMeansClustering(k=no_of_clusters)
cosine_kmeans_labels = cosine_kmeans.fit(cosine_distances, max_iterations=Max_iterations)

In [44]:
jaccard_distances = pairwise_distances(data, metric='hamming')
jaccard_kmeans = KMeansClustering(k=no_of_clusters)
jaccard_kmeans_labels = jaccard_kmeans.fit(X=jaccard_distances, max_iterations=Max_iterations)

### Q1. Run K-means clustering with Euclidean, Cosine and Jarcard similarity. Specify K= the number of categorical values of y (the number of classifications). Compare the SSEs of Euclidean-K-means, Cosine-K-means, Jarcard-K-means. Which method is better? (10 points)

In [45]:
sse_euclidean_m = euclidean_kmeans.get_sum_of_squared_error()
sse_euclidean_m

48177319.88235207

In [46]:
sse_cosine_m = cosine_kmeans.get_sum_of_squared_error()
sse_cosine_m

6791.817912519916

In [47]:
see_jaccard_m = jaccard_kmeans.get_sum_of_squared_error()
see_jaccard_m

1608.2239855646446

In [48]:
text1 = f"SSE of Euclidean K-means = {sse_euclidean_m}<br>"
text2 = f"SSE of Cosine K-means = {sse_cosine_m}<br>"
text3 = f"SSE of Jarcard K-means = {see_jaccard_m}<br>"
display(Markdown(f"{text1}{text2}{text3}"))

SSE of Euclidean K-means = 48177319.88235207<br>SSE of Cosine K-means = 6791.817912519916<br>SSE of Jarcard K-means = 1608.2239855646446<br>

Based on the values above, it can implied that Euclidean K-means has the highest SSE, followed by Cosine K-means and then Jarcard K-means. The one with the lowest SSE is generally considered the better method, here, the Jarcard K-means.

### Q2. Compare the accuracies of Euclidean-K-means Cosine-K-means, Jarcard-K-means. First, label each cluster using the majority vote label of the data points in that cluster. Later, compute the predictive accuracy of Euclidean-K-means, Cosine-K-means, Jarcard-K-means. Which metric is better? (10 points)

In [49]:
def label_clusters(labels, true_labels):
    unique_labels = np.unique(true_labels)
    cluster_labels = np.zeros(len(labels), dtype=int)
    for cluster in range(no_of_clusters):
        cluster_indices = np.where(labels == cluster)[0]
        cluster_true_labels = true_labels[cluster_indices]
        majority_label = np.argmax([np.sum(cluster_true_labels == label) for label in unique_labels])
        cluster_labels[cluster_indices] = majority_label
    return cluster_labels

# Label clusters
cluster_labels_euclidean = label_clusters(euclidean_kmeans_labels, labels)
cluster_labels_cosine = label_clusters(cosine_kmeans_labels, labels)
cluster_labels_jaccard = label_clusters(jaccard_kmeans_labels, labels)

# Compute predictive accuracy
accuracy_euclidean = accuracy_score(labels, cluster_labels_euclidean)
accuracy_cosine = accuracy_score(labels, cluster_labels_cosine)
accuracy_jaccard = accuracy_score(labels, cluster_labels_jaccard)

In [50]:
acc_text1 = f"Accuracy of Euclidean-K-means =  {accuracy_euclidean * 100}%<br>"
acc_text2 = f"Accuracy of Cosine-K-means = {accuracy_cosine * 100}%<br>"
acc_text3 = f"Accuracy of Jaccard-K-means = {accuracy_jaccard * 100}%<br>"
display(Markdown(f"{acc_text1}{acc_text2}{acc_text3}"))

Accuracy of Euclidean-K-means =  59.51%<br>Accuracy of Cosine-K-means = 33.85%<br>Accuracy of Jaccard-K-means = 28.17%<br>

Based on accuracy, it is observed that Euclidean accuracy perform better.

### Q3: Set up the same stop criteria: “when there is no change in centroid position OR when the SSE value increases in the next iteration OR when the maximum preset value (e.g., 500, you can set the preset value by yourself) of iteration is complete”, for Euclidean-K-means, Cosine-K-means, Jarcard-K-means. Which method requires more iterations and times to converge? (10 points)

In [51]:
euclidean_iterations = euclidean_kmeans.get_iterations_to_converge()
cosine_iterations = cosine_kmeans.get_iterations_to_converge()
jacard_iterations = jaccard_kmeans.get_iterations_to_converge()

text_1 = f"Iterations to converge for Euclidean-K-means = **{euclidean_iterations}** <br>"
text_2 = f"Iterations to converge for Cosine-K-means = **{cosine_iterations}** <br>"
text_3 = f"Iterations to converge for Jaccard-K-means = **{jacard_iterations}** <br>"

display(Markdown(f"{text_1}{text_2}{text_3}"))

Iterations to converge for Euclidean-K-means = **93** <br>Iterations to converge for Cosine-K-means = **29** <br>Iterations to converge for Jaccard-K-means = **5** <br>

From the results it can be seen that Euclidean K-means take more iterations to converge.

### Q4: Compare the SSEs of Euclidean-K-means Cosine-K-means, Jarcard-K-means with respect to  the following three terminating conditions: (10 points)
  > - when there is no change in centroid position
  > - when the SSE value increases in the next iteration
  > - when the maximum preset value (e.g., 100) of iteration is complete

##### (a) when there is no change in centroid position

In [52]:
euclidean_kmeans_1 = KMeansClustering(k=no_of_clusters, stopping_criterion="no_change")
euclidean_kmeans_1_predicted_labels = euclidean_kmeans_1.fit(data, max_iterations=Max_iterations)
sse_euclidean_kmeans_1 = euclidean_kmeans_1.get_sum_of_squared_error()
print('SSE of Euclidean K-means when there is no change in centroid position =', sse_euclidean_kmeans_1)

SSE of Euclidean K-means when there is no change in centroid position = 48450382.25619897


In [53]:
cosine_kmeans_1 = KMeansClustering(k=no_of_clusters, stopping_criterion="no_change")
cosine_kmeans_1_predicted_labels = cosine_kmeans_1.fit(cosine_distances, max_iterations=Max_iterations)
sse_cosine_kmeans_1 = cosine_kmeans_1.get_sum_of_squared_error()
print('SSE of Cosine K-means when there is no change in centroid position =', sse_cosine_kmeans_1)

SSE of Cosine K-means when there is no change in centroid position = 2713.7960774918047


In [54]:
jaccard_kmeans_1 = KMeansClustering(k=no_of_clusters, stopping_criterion="no_change")
jaccard_kmeans_1_predicted_labels = jaccard_kmeans_1.fit(jaccard_distances, max_iterations=Max_iterations)
sse_jaccard_kmeans_1 = jaccard_kmeans_1.get_sum_of_squared_error()
print('SSE of Jaccard K-means when there is no change in centroid position =', sse_jaccard_kmeans_1)

SSE of Jaccard K-means when there is no change in centroid position = 1786.7832535384596


##### (b) when the SSE value increases in the next iteration

In [55]:
euclidean_kmeans_2 = KMeansClustering(k=no_of_clusters, stopping_criterion="increase_sse")
euclidean_kmeans_2_predicted_labels = euclidean_kmeans_2.fit(data, max_iterations=Max_iterations)
sse_euclidean_kmeans_2 = euclidean_kmeans_2.get_sum_of_squared_error()
print('SSE of Euclidean K-means when the SSE value increases in the next iteration =',
      sse_euclidean_kmeans_2)

SSE of Euclidean K-means when the SSE value increases in the next iteration = 129016983.95835175


In [56]:
cosine_kmeans_2 = KMeansClustering(k=no_of_clusters, stopping_criterion="increase_sse")
cosine_kmeans_2_predicted_labels = cosine_kmeans_2.fit(cosine_distances, max_iterations=Max_iterations)
sse_cosine_kmeans_2 = cosine_kmeans_2.get_sum_of_squared_error()
print('SSE of Cosine K-means when the SSE value increases in the next iteration =',
      sse_cosine_kmeans_2)

SSE of Cosine K-means when the SSE value increases in the next iteration = 12155.85866595439


In [57]:
jaccard_kmeans_2 = KMeansClustering(k=no_of_clusters, stopping_criterion="increase_sse")
jaccard_kmeans_2_predicted_labels = jaccard_kmeans_2.fit(jaccard_distances, max_iterations=Max_iterations)
sse_jaccard_kmeans_2 = jaccard_kmeans_2.get_sum_of_squared_error()
print('SSE of Jaccard K-means when the SSE value increases in the next iteration =',
      sse_jaccard_kmeans_2)

SSE of Jaccard K-means when the SSE value increases in the next iteration = 2349.8999627724443


##### (c) when the maximum preset value (e.g., 100) of iteration is complete

In [58]:
euclidean_kmeans_3 = KMeansClustering(k=no_of_clusters, stopping_criterion="max_iterations")
euclidean_kmeans_3_predicted_labels = euclidean_kmeans_3.fit(data, max_iterations=Max_iterations)
sse_euclidean_max_iteration = euclidean_kmeans_3.get_sum_of_squared_error()
print(f'SSE of Euclidean K-means when the maximum preset value {Max_iterations} is complete =',
      sse_euclidean_max_iteration)

SSE of Euclidean K-means when the maximum preset value 100 is complete = 56618625.4482222


In [59]:
cosine_kmeans_3 = KMeansClustering(k=no_of_clusters, stopping_criterion="max_iterations")
cosine_kmeans_3_predicted_labels = cosine_kmeans_3.fit(cosine_distances, max_iterations=Max_iterations)
sse_cosine_max_iteration = cosine_kmeans_3.get_sum_of_squared_error()
print(f'SSE of Cosine K-means when the maximum preset value {Max_iterations} is complete =',
      sse_cosine_max_iteration)

SSE of Cosine K-means when the maximum preset value 100 is complete = 4787.641689948039


In [60]:
jaccard_kmeans_3 = KMeansClustering(k=no_of_clusters, stopping_criterion="max_iterations")
jaccard_kmeans_3_predicted_labels = jaccard_kmeans_3.fit(jaccard_distances, max_iterations=Max_iterations)
sse_jarcard_max_iteration = jaccard_kmeans_3.get_sum_of_squared_error()
print(f'SSE of Jaccard K-means when the maximum preset value {Max_iterations} is complete =',
      sse_jarcard_max_iteration)

SSE of Jaccard K-means when the maximum preset value 100 is complete = 1822.2396663347374


In [61]:
table = f"""
| Algorithm | No Change in Centroid Position | SSE Value Increases in Next Iteration | Maximum Preset Value of Iterations |
|------------|--------------------------------|----------------------------------------|-------------------------------------|
| Euclidean  | {sse_euclidean_kmeans_1}              | {sse_euclidean_kmeans_2}                    | {sse_euclidean_max_iteration}|
| Jaccard    |{sse_jaccard_kmeans_1}          |{sse_jaccard_kmeans_2}                  |{sse_jarcard_max_iteration}        |
| Cosine     |{sse_cosine_kmeans_1}           |{sse_cosine_kmeans_2}                   |{sse_cosine_max_iteration}         |
"""
display(Markdown(table))



| Algorithm | No Change in Centroid Position | SSE Value Increases in Next Iteration | Maximum Preset Value of Iterations |
|------------|--------------------------------|----------------------------------------|-------------------------------------|
| Euclidean  | 48450382.25619897              | 129016983.95835175                    | 56618625.4482222|
| Jaccard    |1786.7832535384596          |2349.8999627724443                  |1822.2396663347374        |
| Cosine     |2713.7960774918047           |12155.85866595439                   |4787.641689948039         |


### Q5: What are your summary observations or takeaways based on your algorithmic analysis?

- Based on the values from the above table, it can be observed that Euclidean K-means has the highest SSE, followed by Cosine K-means and then the Jacard K-means with the lowest SSE.
- It can also be noted that Euclidean K-means take more iterations to converge.
- Since Jaccard has lower SSE value, it is the better method.

